# Libraries used

In [53]:
import pandas as pd

# Importing the DF
- We have a train and test csv for comparason and also a reference for how it should look at the end (sample_submission)

In [56]:
df_train = pd.read_csv('csvs/train.csv') 
df_test = pd.read_csv('csvs/test.csv')
sample_submission = pd.read_csv('csvs/sample_submission.csv')

In [58]:
df_train.head(5)

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10


In [60]:
df_test.head(5)

,id,date,store,item
0,0,2018-01-01,1,1
1,1,2018-01-02,1,1
2,2,2018-01-03,1,1
3,3,2018-01-04,1,1
4,4,2018-01-05,1,1
